# L5: Inference

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

# Answer Encoder Used During Ingest Flow

## Text extraction
## Chunking
## Answer Encoder
## Vector DB
### user query --> question encoder --> retrieval (with access to the DB) --> facts --> LLM

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 11.2 MB/s eta 0:00:00


In [4]:
import numpy as np

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, \
                         DPRContextEncoder, DPRQuestionEncoder

import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

# inference production in similarity serach

# neural / vector search

## can't compute similarity against all chuncks

# sim(1, a(i))
# ANN

## HNSW, annoy, faiss, nmsib, are all algoirthms and systems that implement ANN

# in production, you need to add persistence, moving the implementation of these algos from in memory to on disk vector stores.

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> file:</b> To access <code>requirements.txt</code> for this notebook, 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [5]:
# just as before
def cosine_similarity_matrix(features):
    norms = np.linalg.norm(features, axis=1, keepdims=True)
    normalized_features = features / norms
    similarity_matrix = np.inner(normalized_features, normalized_features)
    rounded_similarity_matrix = np.round(similarity_matrix, 4)
    return rounded_similarity_matrix

## Pure similarity

In [6]:
answers = [
    "What is the tallest mountain in the world?",
    "The tallest mountain in the world is Mount Everest.",
    "Mount Shasta",
    "I like my hike in the mountains",
    "I am going to a yoga class"
]

question = 'What is the tallest mountain in the world?'

In [7]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
question_embedding = list(model.encode(question))

sim = []
for answer in answers:
    answer_embedding = list(model.encode(answer))
    sim.append(cosine_similarity_matrix(np.stack([question_embedding, answer_embedding]))[0,1])

print(sim)
best_inx = np.argmax(sim)
print(f"Question = {question}")
print(f"Best answer = {answers[best_inx]}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[1.0, 0.7976, 0.4001, 0.3559, 0.0972]
Question = What is the tallest mountain in the world?
Best answer = What is the tallest mountain in the world?


## Dual-Encoder inference

In [10]:
# using hugging face to download directly

from transformers import AutoTokenizer, DPRContextEncoder, DPRQuestionEncoder

# Load the tokenizer and model from Hugging Face model hub
answer_tokenizer = AutoTokenizer.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
answer_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")

question_tokenizer = AutoTokenizer.from_pretrained("facebook/dpr-question_encoder-multiset-base")
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-multiset-base")


# answer_tokenizer = AutoTokenizer \
#                    .from_pretrained("./models/facebook/dpr-ctx_encoder-multiset-base")
# answer_encoder = DPRContextEncoder \
#                    .from_pretrained("./models/facebook/dpr-ctx_encoder-multiset-base")

# question_tokenizer = AutoTokenizer \
#                    .from_pretrained("./models/facebook/dpr-question_encoder-multiset-base")
# question_encoder = DPRQuestionEncoder \
#                    .from_pretrained("./models/facebook/dpr-question_encoder-multiset-base")


In [12]:
# Compute the question embeddings
question_tokens = question_tokenizer(question, return_tensors="pt")["input_ids"]
question_embedding = question_encoder(question_tokens).pooler_output.flatten().tolist()
print(question_embedding[:10]
      ,len(question_embedding))

[0.07758235931396484, 0.25172534584999084, 0.18663953244686127, 0.22120122611522675, 0.02641540765762329, -0.15785618126392365, 0.32760271430015564, 0.26732850074768066, -0.0850304588675499, 0.1292945295572281] 768


In [13]:
sim = []
for answer in answers:
    answer_tokens = answer_tokenizer(answer, return_tensors="pt")["input_ids"]
    answer_embedding = answer_encoder(answer_tokens).pooler_output.flatten().tolist()
    sim.append(cosine_similarity_matrix(np.stack([question_embedding, answer_embedding]))[0,1])

print(sim)
best_inx = np.argmax(sim)
print(f"Question = {question}")
print(f"Best answer = {answers[best_inx]}")
# exactly what we wanted

[0.6253, 0.7472, 0.5506, 0.3687, 0.25]
Question = What is the tallest mountain in the world?
Best answer = The tallest mountain in the world is Mount Everest.


# Rag workflow in images